This is the cretin main file where all the functionality is available. To use this file succesfully you must have added your paths to the paths.py file and you must have cretin installed which is coupled to via the demo.sh.
I will the showcase the full functionality in this file  

In [6]:
from importlib import reload
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file #these python files should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file]:
    reload(obj)

import numpy as np

In [7]:
# These are all the functions inside the generator object class
gen_object = [i for i in dir(generator_object.User_input()) if i[0:2] != '__']
print(gen_object)

# all the function inside the to_generator file 
gen_to_file = [i for i in dir(to_generator_string.Text_generator(generator_object.User_input())) if i[0:2] != '__']
print(gen_to_file)

['add_plot', 'atoms', 'controls', 'controls_history', 'controls_history_tv', 'drop', 'geometry', 'geometry_nodes', 'geometry_product_mesh', 'geometry_quad', 'lasers', 'materials_atom', 'materials_atom_modeltype', 'materials_region', 'materials_region_background', 'materials_region_element', 'materials_region_level', 'materials_region_material', 'materials_region_opacity', 'materials_region_rho', 'other_switches', 'parameters', 'plots', 'popular_switches', 'radiation_angles', 'radiation_aprd', 'radiation_ebins', 'radiation_lbins', 'radiation_line', 'radiation_spectrum', 'regions', 'source_histories', 'sources', 'sources_aprd', 'sources_boundary', 'sources_history', 'sources_laser', 'sources_lasray', 'sources_rswitch', 'sources_source_jbndry', 'sources_source_jnu', 'sources_source_laser']
['controls', 'dict', 'edits', 'execute', 'flat', 'geometry', 'ilts', 'materials', 'pop_switches', 'radiation', 'sources', 'start_chapter', 'user_input']


In [8]:
# This allows us to find frequntly used terms in the generator files, 
# To be specific the lines of text in the generator files get sliced on " " and then selected by the input, f.e 'alias'
# Then a histogram gets made of the most common results
# similarly for most used words in a chapter (ie "Radiation", "Geography" etc).
# i'm using this to determine what syntax i should include in this python version

search_for_word = 'alias'
search_inside_chapter = 'Controls'
df = search.Generators().word(input = search_for_word, word_slicing = (0,10), print_path = False)
#print(df.to_string())
#print(search.Generators().chapter(input = search_inside_chapter))


In [9]:
# reproducing detecterz.gen
gen1 = generator_object.User_input()

k1, k2 = 1, 21
l1, l2 = 1, 11
n2 = (k2*l2)/2 
NI, NE, TE = 10**20, 10**20, 10**3
R0, R1, Z0, Z1  = 0. , 1., -5, 5
RD, ZD = R1 + .1, Z1 +.1
MU1 = 10**-3
MU2 = -1 + MU1
PHI1, PHI2, PHI3 = 1.57, .785, .524

gen1.materials_region(nodes = [k1, k2, l1, l2], elec_temp = 1.1)
gen1.materials_region_background(k1, k2, l1, l2, TE)
gen1.geometry(type = 'rz')
gen1.geometry_quad(node_1 = [k1, k2], node_2= [l1, l2], x_cors = [R0, R1], y_cors = [Z0, Z1], ratios = [1,1])
gen1.radiation_ebins(n_boundaries = 50, start = 100, end = 1000)
gen1.radiation_angles(n_rays = 3, n_angles = 15)
gen1.controls(t_start = 0, t_end = 0)
gen1.popular_switches(continuum_transfer='do steady-state continuum transfer', include_degeneracy='include electron degeneracy')

# this automatically creates the 'trial' folder, writes to the 'trial.gen' file, runs cretin, extracts the data and plots it into the trial/images folder 
write_run_plot.all(name = 'trial1', object = gen1, longprint = False, plot_duplicates = False)


writing to /home/brewster/Desktop/bowie/trial1/trial1.gen
running cretin with trial1
file list ['trial1.d00']
trial1 /home/brewster/Desktop/bowie//trial1/trial1.d00
plotting trial1 to /home/brewster/Desktop/bowie/trial1/images
emis_0 has shape (11, 21, 50) and has not been plot
jnu_0 has shape (11, 21, 50) and has not been plot
kappa_0 has shape (11, 21, 50) and has not been plot


In [11]:
# Here we reproduce lasfoam.gen from the test folder

def lasfoam(source):

    gen2 = generator_object.User_input()

    N0, N1 = 1, 21 #number of nodes is 20
    DN, DR = 10, .1
    E1, E2 = 4.68, 4.69
    TE, rho = 2.0, 0.003
    E1, E2 = 4.68, 4.68*1.001
    HPEV = 4.135667516e-15
    mult =  HPEV / (E2 - E1)
    RSPOT = .1
    ELASER, TLASER = 1000, 2e-9
    ILASER = 10**7*ELASER/(TLASER*np.pi*RSPOT**2)
    
    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "h")    
    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "c")

    gen2.materials_region(nodes = [N0, N1], elec_temp = TE)
    gen2.materials_region_rho(rho)
    gen2.materials_region_element(index = 1, initial_ion_population = 1.)
    gen2.materials_region_element(index = 2, initial_ion_population = 1.)

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end = 3e-9, restart = True)
    gen2.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen2.controls_history_tv(time = 0., value = 1)
    gen2.controls_history_tv(time = 1.0, value = 1.)
    gen2.controls_history_tv(time = 1.0, value = 0.)

    gen2.parameters(scattering_muliplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-14, maximum_timestep = 10**-10, time_between_snapshots=10**(-9))
    gen2.popular_switches(temparture_calc_heating_rates=['temp calc = time dependant','heating rates = electronic'], max_iterations_per_timestep=10)

    if source == 'SOURCE1':
        gen2.sources_source_laser(laser_wavelength = 4, option_1= 'rate', option_2= 'history', values = [1,1], nodes= [N0, N1])

    elif source == 'SOURCE2':
        gen2.sources_source_jnu(E_range=[E1,E2], option_1= 'rate', option_2= 'history', values = [1,mult], nodes= [N0, N1])

    elif source == 'SOURCE3':
        gen2.sources_source_jbndry(index = 1, E_range= [E1, E2], option_1= 'rate', option_2= 'history',values = [1,mult])
        gen2.sources_boundary(package='radiation', type = 'streaming', nodes= [1, 0, N1, 'xfile'], value = 1, mult = 1)
        gen2.popular_switches(temparture_calc_heating_rates=['temp calc = time dependant','heating rates = electronic'], 
                              max_iterations_per_timestep = 10, continuum_transfer='do steady-state and use integral formalism formalism')

    elif source == 'SOURCE4':
        gen2.sources_source_jbndry(index = 1, E_range= [E1, E2], option_1= 'rate', option_2= 'history',values = [1,mult])
        gen2.sources_boundary(package='radiation', type = 'streaming', nodes= [1, 0, N1, 'xfile'], value = 1, mult = 1)
        gen2.popular_switches(continuum_transfer_evolves_temp = True)
        gen2.sources_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)  
    
    return gen2


gen2 = lasfoam('SOURCE4')
name = 'checking_lasfoam4'
write_run_plot.all(name, gen2, longprint=False, plot_duplicates= False)


AttributeError: 'User_input' object has no attribute 'lasray_lis'

In [ ]:
# here we plot mutiple simulations into one plot to compare them
# first we create our generic object
def sim(var):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin = 6.9, 30*var, 512,  
    ne, z = 10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 40 nodes
    tin_n = 50
    E1, E2 = 4, 4.7 # the laser energies
    DN, DR = 10, .1

    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 121, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end=3*10**-6, restart = True)
    gen2.popular_switches(timestep_between_snapshot = 5)
    return gen2


i0, iend = 0,6
trials = []
name = 'trial'
# Here i showcase the mutiplot functionality automatically putting all the data into one plot per type
for i in range(i0,iend):

    trial = name+str(i)
    trials.append(trial)
    variable = 2**i
    gen2 = sim(variable)
    write_run_plot.write(name = name+str(i), object = gen2)
    write_run_plot.run(name = name+str(i), longprint=False)

serial_sim_tools.plot_all('trials_mutiplot', trials)



writing to /home/brewster/Desktop/bowie/trial0/trial0.gen
running cretin with trial0

writing to /home/brewster/Desktop/bowie/trial1/trial1.gen
running cretin with trial1

writing to /home/brewster/Desktop/bowie/trial2/trial2.gen
running cretin with trial2

writing to /home/brewster/Desktop/bowie/trial3/trial3.gen
running cretin with trial3

writing to /home/brewster/Desktop/bowie/trial4/trial4.gen
running cretin with trial4

writing to /home/brewster/Desktop/bowie/trial5/trial5.gen
running cretin with trial5
multiplot to /home/brewster/Desktop/bowie/trials_mutiplot


In [ ]:
def sim1(var1, var2, var3):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin = 6.9*var1, 30*var2, 512*var3,  
    ne, z = 10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 40 nodes
    tin_n = 50
    E1, E2 = 4, 4.7 # the laser energies
    DN, DR = 10, .1


    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 121, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end=3*10**-6, restart = True)
    gen2.popular_switches(timestep_between_snapshot = 5)
    return gen2

# Here i showcase the combinatorcs functionality allowin the combination of any set of paramters for mutiplot
vars1, vars2, vars3 = np.arange(1,3,1), np.arange(4,6,1), np.arange(2,12,5)
arr = np.array([vars1, vars2, vars3], dtype=object)
combinations = serial_sim_tools.combinatorics(arr = arr)
names = []
base_name = 'combinatorics_sim'

for combination in combinations:
    [var1,var2,var3] = combination
    gen2 = sim1(var1, var2, var3)
    name = f'{base_name}_{var1}_{var2}_{var3}'
    names.append(name)
    write_run_plot.write(name = name, object = gen2)
    write_run_plot.run(name = name, longprint=False)

serial_sim_tools.plot_all(f'{base_name}_multiplot', names)


writing to /home/brewster/Desktop/bowie/combinatorics_sim_1_4_2/combinatorics_sim_1_4_2.gen
running cretin with combinatorics_sim_1_4_2

writing to /home/brewster/Desktop/bowie/combinatorics_sim_1_4_7/combinatorics_sim_1_4_7.gen
running cretin with combinatorics_sim_1_4_7

writing to /home/brewster/Desktop/bowie/combinatorics_sim_1_5_2/combinatorics_sim_1_5_2.gen
running cretin with combinatorics_sim_1_5_2

writing to /home/brewster/Desktop/bowie/combinatorics_sim_1_5_7/combinatorics_sim_1_5_7.gen
running cretin with combinatorics_sim_1_5_7

writing to /home/brewster/Desktop/bowie/combinatorics_sim_2_4_2/combinatorics_sim_2_4_2.gen
running cretin with combinatorics_sim_2_4_2

writing to /home/brewster/Desktop/bowie/combinatorics_sim_2_4_7/combinatorics_sim_2_4_7.gen
running cretin with combinatorics_sim_2_4_7

writing to /home/brewster/Desktop/bowie/combinatorics_sim_2_5_2/combinatorics_sim_2_5_2.gen
running cretin with combinatorics_sim_2_5_2

writing to /home/brewster/Desktop/bowie/

In [ ]:
# Here i showcase the ability to add plots, and grouping together plots into diffrent multiplot files

def tin(source, variable):

    gen2 = generator_object.User_input()

    N0, N1 = 1, 21 #number of nodes is 20
    DR =  .1
    TE, rho = 30.0, .002
    E1, E2 = 4.68, 4.68*1.001
    HPEV = 4.135667516e-15
    mult =  HPEV / (E2 - E1)
    RSPOT = .01
    ELASER, TLASER = .06, 2e-9
    ILASER = variable*10**7*ELASER/(TLASER*np.pi*RSPOT**2)
    
    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "sn")    

    gen2.materials_region(nodes = [N0, N1], elec_temp = TE)
    gen2.materials_region_rho(rho)
    gen2.materials_region_element(index = 1, initial_ion_population = 1.)


    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 101, start = 0.1, end = 10**(2))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end = 3e-9, restart = True)
    gen2.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen2.controls_history_tv(time = 0., value = 1)
    gen2.controls_history_tv(time = 1.0, value = 1.)
    gen2.controls_history_tv(time = 1.0, value = 0.)


    gen2.parameters(scattering_muliplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-14, maximum_timestep = 10**-9, time_between_snapshots=10**(-9))
    gen2.popular_switches(temparture_calc_heating_rates=['temp calc = time dependant','heating rates = electronic'], max_iterations_per_timestep=20)

    gen2.add_plot(name = 'plotter', xvar = 'time', yvar = 'tiv')
    gen2.add_plot(name = 'plotter1', xvar = 'time', yvar = 'tev')
    gen2.add_plot(name = 'plotter2', xvar = 'time', yvar = 'rho')
    gen2.add_plot(name = 'plotter3', xvar = 'time', yvar = 'tev',element_or_transition='1:N:DN')
    gen2.add_plot(name = 'plotter4', xvar = 'time', yvar = 'tautot')
    gen2.add_plot(name = 'plotter5', xvar = 'time', yvar = 'zbar')
    gen2.add_plot(name = 'plotter6', xvar = 'time', yvar = 'nion')
    gen2.add_plot(name = 'plotter7', xvar = 'time', yvar = 'ne')

    if source == 'SOURCE1':
        gen2.source_laser(laser_wavelength = 4, option_1= 'rate', option_2= 'history', values = [1,1], nodes= [N0, N1])

    elif source == 'SOURCE2':
        gen2.source_jnu(E_range=[E1,E2], option_1= 'rate', option_2= 'history', values = [1,mult], nodes= [N0, N1])

    elif source == 'SOURCE3':
        gen2.source_jbndry(index = 1, E_range= [E1, E2], option_1= 'rate', option_2= 'history',values = [1,mult])
        gen2.source_boundary(package='radiation', type = 'streaming', nodes= [1, 0, N1, 'xfile'], value = 1, mult = mult)
        gen2.popular_switches(temparture_calc_heating_rates=['temp calc = time dependant','heating rates = electronic'], 
                              max_iterations_per_timestep = 10, continuum_transfer='do steady-state and use integral formalism formalism')

    elif source == 'SOURCE4':
        gen2.source_jbndry(index = 1, E_range= [E1, E2], option_1= 'rate', option_2= 'history',values = [1,mult])
        gen2.source_boundary(package='radiation', type = 'streaming', nodes= [1, 0, N1, 'xfile'], value = 1, mult = mult)
        gen2.popular_switches(continuum_transfer_evolves_temp = True)
        gen2.source_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)  
    
    return gen2

i0 = 1
trials = []
name = 'trial'
N, M = 3, 3 # this is the multiplot group size and number of groups
iend = i0+M*N

for i in range(i0,iend):
    name_i = name+str(i)
    trials.append(name_i)
    variable = i
    gen2 = tin('SOURCE1', variable)

    write_run_plot.write(name = name_i, object = gen2)
    write_run_plot.run(name = name_i, longprint=False)

    if (i>0 and i%N == 0) or i == iend -1:
        serial_sim_tools.plot_all(f'trials_mutiplot{int(i/N)}', trials)
        trials = []



including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

writing to /home/brewster/Desktop/bowie/trial1/trial1.gen
running cretin with trial1
including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

writing to /home/brewster/Desktop/bowie/trial2/trial2.gen
running cretin with trial2
including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

writing to /home/brewster/Desktop/bowie/trial3/trial3.gen
running cretin with trial3
multiplot to /home/brewster/Desktop/bowie/trials_mutiplot1
including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

wr

In [ ]:
from importlib import reload
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file, generator_object_dropdowns #these python files should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file, generator_object_dropdowns]:
    reload(obj)

gen3 = generator_object_dropdowns.User_input()
functions = [i for i in dir(gen3) if callable(getattr(gen3, i)) and not i.startswith("__") and ("drop" in i)]
print(functions)
for method_name in functions:
    method_to_call = getattr(gen3, method_name)
    #method_to_call()



gen3.drop_source_lasray()


['drop_add_plot', 'drop_geometry', 'drop_geometry_nodes', 'drop_geometry_product_mesh', 'drop_geometry_quad', 'drop_materials_atom', 'drop_materials_atom_modeltype', 'drop_materials_region', 'drop_materials_region_background', 'drop_materials_region_element', 'drop_materials_region_level', 'drop_materials_region_material', 'drop_materials_region_opacity', 'drop_materials_region_rho', 'drop_parameters', 'drop_popular_switches', 'drop_radiation_angles', 'drop_radiation_ebins', 'drop_radiation_lbins', 'drop_radiation_line', 'drop_source_boundary', 'drop_source_jbndry', 'drop_source_jnu', 'drop_source_laser', 'drop_source_lasray', 'drop_source_rswitch']


In [ ]:
gen3.source_boundaries

['boundary', ['radiation', 'flux', [1, 2, 3, 4], 3.0, 3.0], []]

In [ ]:
#TODO
# check functionality of all region commands
# add lasray functionality
# turn dropdown functions into regular functions using chatgpt

In [ ]:
search_for_word = 'lasray'
search_inside_chapter = 'Controls'
df = search.Generators().word(input = search_for_word, word_slicing = (0,10), print_path = False)
print(df.to_string())

                             0
lasray 0.11 1. 0. 1.         4
lasray RLASER 1. 0. 1. FABS  1


In [ ]:
#lasray testing


gen3.drop_source_lasray()

gen3.drop_source_lasray()
gen3.drop_source_lasray()

gen3.drop_source_laser()
gen3.drop_source_lasray()

In [ ]:
from importlib import reload
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file, generator_object_dropdowns #these python files should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file, generator_object_dropdowns]:
    reload(obj)

#gen3 = generator_object_dropdowns.User_input()


